In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from bqplot import pyplot as plt
import numpy as np
from scipy.integrate import odeint
import pandas as pd
from scipy.optimize import curve_fit

In [2]:
N = 5.46e6
t = np.linspace(0,70,70)

data = pd.read_csv('minnesota_data.csv')
good_days = data.index >= 210

confirmed = data.Confirmed
recovered = data.Recovered
death = data.Deaths
sus = N-confirmed - recovered - death
active = data.Active

In [3]:
def calc(b,g):
    I0, R0 = 12817,114679
    S0 = N - I0 - R0
    beta, gamma = b, g 

    def deriv(y, t, N, beta, gamma):
        S, I, R = y
        dSdt = -beta * S * I / N
        dIdt = beta * S * I / N - gamma * I
        dRdt = gamma * I
        return dSdt, dIdt, dRdt

    y0 = S0, I0, R0
    ret = odeint(deriv, y0, t, args=(N, beta, gamma))
    return ret.T

def graph(b,g):
    S,I,R= calc(b,g)
    fig.marks[0].y = S/N
    fig.title=f'Susceptible, contact: {b*100:.2f}%, recovery: {g*100:.2f}%'

In [4]:
fig = plt.figure(1, title='Something',animation_speed=1000)
S,I,R= calc(0.90,0.80)
plt.plot(t, S/N, 'r--', alpha=0.5, lw=2, labels=['SIR'])
index = data.index[good_days]
plt.plot(index[60:]-data.index[good_days][60],sus[good_days][60:]/N,'b-',labels=['Actual Data'])
plt.legend()
plt.show()

interact(graph,b=(0.1,1.0,0.01),g=(0.1,1.0,0.01))

interactive(children=(FloatSlider(value=0.55, description='b', max=1.0, min=0.1, step=0.01), FloatSlider(value…

<function __main__.graph(b, g)>

In [5]:
def h(t,b,g):
    S,I,R = calc(b,g)
    return S/N

x = t
y = sus[good_days][60:130]/N
popt, pcov = curve_fit(h, x, y)
popt

array([0.57870143, 0.53099223])

In [6]:
perr = np.sqrt(np.diag(pcov))
perr

array([0.01948416, 0.01925093])